In [1]:
import sys
sys.path.insert(0, './gluon-cv/')

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import mxnet as mx
from mxnet import ndarray as F
from models.generator import defineG_unet
from models.discriminator import defineD_n_layers
from mxnet import gluon

from datasets.fascades import Fascades, _transformer

In [2]:
dataset = Fascades(
    root='/home/chenyifeng/pix2pix/datasets/facades/', 
    split='train', 
    mode='train', 
    transform=_transformer(),
    crop_size=256)

ngpus = len(mx.test_utils.list_gpus())
batch_size = 8 * ngpus
loader = gluon.data.DataLoader(dataset, batch_size, shuffle=True, last_batch='rollover',num_workers=4)

In [3]:
load_iter = loader.__iter__()
image, label = load_iter.next()

gen = defineG_unet(input_nc=3, output_nc=3, ngf=64, ctx=mx.gpu(0))
dis = defineD_n_layers(input_nc=3, output_nc=3, ndf=64, n_layers=3, ctx=mx.gpu(0))

from gluoncv.utils.parallel import split_and_load
from parallel import DataParallelCriterionAug, DataParallelModelAug
from gluoncv.model_zoo.segbase import SegEvalModel

gen_net = DataParallelModelAug(gen, [mx.gpu(i) for i in range(ngpus)] , True)
dis_net = DataParallelModelAug(dis, [mx.gpu(i) for i in range(ngpus)] , True)
# criterion =  DataParallelCriterion(_criterion, [mx.gpu(i) for i in range(ngpus)] , True)
# evaluator = DataParallelModel(SegEvalModel(model), [ mx.gpu(i) for i in range(ngpus)])

In [4]:
from gluoncv.utils.lr_scheduler import LRScheduler

base_lr = 2e-4
nepochs = 200
scheduler = LRScheduler(mode='poly',
                        base_lr=base_lr, 
                        target_lr=0, 
                        nepochs=nepochs, 
                        iters_per_epoch=len(loader), 
                        power=0.9)


optimizer_params = {
    'learning_rate':base_lr,
    'wd': 0.0,
    'lr_scheduler': scheduler,
    'momentum': 0.9
}



gen_opt = mx.gluon.Trainer(gen.collect_params(), 'sgd', optimizer_params, kvstore=mx.kv.create('device'))
dis_opt = mx.gluon.Trainer(dis.collect_params(), 'sgd', optimizer_params, kvstore=mx.kv.create('device'))

In [5]:
from mxnet import autograd
from criterion import GeneratorCriterion, DiscriminatorCriterion

_GeneratorCriterion = GeneratorCriterion(weight=100)
_DiscriminatorCriterion = DiscriminatorCriterion()

g_criterion = DataParallelCriterionAug(_GeneratorCriterion, [mx.gpu(i) for i in range(ngpus)] , True)
d_criterion = DataParallelCriterionAug(_DiscriminatorCriterion, [mx.gpu(i) for i in range(ngpus)] , True)

In [6]:
from datetime import datetime
from tqdm import tqdm_notebook
from mxboard import SummaryWriter
sw = SummaryWriter(logdir='/tmp/mx_logs/{}'.format(datetime.now().strftime('%Y_%m_%d_%H:%M:%S')), flush_secs=5, verbose=False)

In [7]:
add_tuple_inside = lambda x: [ (i,) for i in x]

In [8]:
epoch = 0
for _ in range(nepochs):
    tbar = tqdm_notebook(loader)
    epoch += 1
    total_dloss = 0
    total_gloss = 0
    for idx, (image, label) in enumerate(tbar):                             
        real_A = label
        real_B = image
        
        real_AB = F.concat(*[real_A, real_B], dim=1)
        real_A = split_and_load(real_A, [mx.gpu(i) for i in range(ngpus)])
        real_B = split_and_load(real_B, [mx.gpu(i) for i in range(ngpus)])

        with autograd.record():
            fake_B = gen_net._split_call(add_tuple_inside(real_A))
            fake_AB = [ (F.concat(*[i, j[0]], dim=1),) for (i, j) in zip(real_A, fake_B)]

            output_fake = dis_net._split_call(fake_AB)
            output_real = dis_net(real_AB)
            dloss = d_criterion(output_real, output_fake)
            autograd.backward(dloss)
            mx.nd.waitall()
        dis_opt.step(batch_size)
        for loss in dloss:
            total_dloss += loss.asnumpy()[0] / len(dloss)

        with autograd.record():
            fake_B = gen_net._split_call(add_tuple_inside(real_A))   
            fake_AB = [ (F.concat(*[i, j[0]], dim=1),) for (i, j) in zip(real_A, fake_B)]
            
            output_fake = dis_net._split_call(fake_AB)
            fake_real_B = [ (i[0], j) for (i, j) in zip(fake_B, real_B)]
            gloss = g_criterion(output_fake, fake_real_B)
            autograd.backward(gloss)
            mx.nd.waitall()
        gen_opt.step(batch_size)
        for loss in gloss:
            total_gloss += loss.asnumpy()[0] / len(gloss)                             

        tbar.set_description_str('Epoch {}: dloss : {:.4f} gloss : {:.4f}'.format(epoch, total_dloss/(idx+1), total_gloss/(idx+1)))
    
    if epoch % 50 == 0:
        gen_net.module.save_parameters('/home/chenyifeng/MxData/models/gan/fascades/Gen_{}.params'.format(epoch))
        dis_net.module.save_parameters('/home/chenyifeng/MxData/models/gan/fascades/Dis_{}.params'.format(epoch))
    
    sw.add_scalar(tag='dloss_epoch', value=total_dloss/(idx+1), global_step=epoch)
    sw.add_scalar(tag='gloss_epoch', value=total_gloss/(idx+1), global_step=epoch)